In [ ]:
 %load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

from skorecard import datasets
from skorecard.preprocessing import TreeBucketTransformer

In [ ]:
df = datasets.load_uci_credit_card(as_frame=True)
df.head(3)

## Test one feature

In [ ]:
X = df['LIMIT_BAL'].values
y = df['default'].values

In [ ]:
from skorecard.preprocessing import TreeBucketTransformer

In [ ]:
tbt = TreeBucketTransformer(inf_edges=False, 
    max_depth=4,
   criterion = 'entropy',
   min_samples_leaf =0.04 , #Minimum number of entries in the bins
   min_impurity_decrease=0.0001)

In [ ]:
tbt.get_params()

In [ ]:
tbt.fit(X, y)

In [ ]:
tbt.fit(X, y)

In [ ]:
tbt.predict(X)

### Optimize within the sklearn api

In [ ]:
from skorecard.preprocessing.woe import woe_1d
from skorecard.metrics.metrics import IV_scorer
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    'max_depth':[3,4],
    'min_samples_leaf':[0.05,0.06, 0.07, 0.08],
    'min_impurity_decrease':[0.0001,0.0002,0.0003, 0.0004, 0.0005 ]
}

In [ ]:
gs = GridSearchCV(
    estimator = TreeBucketTransformer(),
    scoring = IV_scorer,
    cv=3,
    param_grid = parameters,
    return_train_score = True
)

In [ ]:
gs.fit(X,y)

In [ ]:
cv_res = pd.DataFrame(gs.cv_results_)

In [ ]:
(
    cv_res.sort_values(by='mean_train_score', ascending=False) 
    [['mean_train_score','mean_test_score',"param_max_depth","param_min_impurity_decrease","param_min_samples_leaf"]]
    .head(10)
)

In [ ]:
cv_res.loc[20,'params']

## Get the best estimator

In [ ]:
tbt_best = gs.best_estimator_

In [ ]:
tbt_best.boundaries

In [ ]:
tbt_best.get_params()

In [ ]:
buckets = pd.DataFrame({'bins':tbt_best.transform(X), 'y':y})

# Plots
* default rate in bucket
* count per bucket (in blue)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax2 =ax.twinx()
buckets.groupby("bins").agg("mean")['y'].plot(ax=ax, color='red')
buckets.groupby("bins").agg("count")['y'].plot(ax=ax2, color='blue')